In [1]:
#가상환경 python 3.9 새로 설치해서 진행
# !pip install torch
# !pip install konlpy
# !pip install numpy pandas
# !pip install tqdm
# !pip install torchtext
# !pip install scikit-learn

In [1]:
import konlpy
konlpy.data.path

['C:\\Users\\user/konlpy_data',
 'C:\\konlpy_data',
 'D:\\konlpy_data',
 'E:\\konlpy_data',
 'c:\\ProgramData\\Anaconda3\\konlpy_data',
 'c:\\ProgramData\\Anaconda3\\lib\\konlpy_data',
 'C:\\Users\\user\\AppData\\Roaming\\konlpy_data',
 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\konlpy/data']

In [2]:
import pandas as pd
import os
import re
import random
from konlpy.tag import Mecab
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
# torch==1.10.0
# torchtext==0.11.2
# legacy 사용해서 코드 진행하기 위해 버전 맞춰줬음!

import torch
import torchtext
from torchtext.legacy import data, datasets
from torchtext.legacy.data import TabularDataset

import torch.nn as nn
import torch.nn.functional as F


In [3]:
# 시드 고정
SEED = 10
random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
df_train = pd.read_csv("./input/dataset/train.csv")
print(len(df_train)) #52682 train, 13181 valid로 분할 후 저장
df_train.head()

65863


,ID,text,label
0,0,유소영비호감 성형아줌마,1
1,1,나오지마라 썅,3
2,2,식상하고 지긋지긋했는데 잘 끝나네 오예 소리벗고 빤스질러~~!!!,6
3,3,성희롱 당할 얼굴이 아닌데?ㅋㅋㅋ,5
4,4,"""끝까지 해보자~쪽파리 원숭이 자한 쓰레기당""",0


In [5]:
for i in range(len(df_train)):
    df_train["text"][i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ ]","",df_train["text"][i]) #한국어 자음 포함해서 남겨둔 상태

# df_train = df_train[['text','label']]
df_train.head()

C:\Users\user\AppData\Local\Temp\ipykernel_11572\1472338467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["text"][i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ ]","",df_train["text"][i]) #한국어 자음 포함해서 남겨둔 상태


,ID,text,label
0,0,유소영비호감 성형아줌마,1
1,1,나오지마라 썅,3
2,2,식상하고 지긋지긋했는데 잘 끝나네 오예 소리벗고 빤스질러,6
3,3,성희롱 당할 얼굴이 아닌데ㅋㅋㅋ,5
4,4,끝까지 해보자쪽파리 원숭이 자한 쓰레기당,0


In [6]:
#52683 train, 13180 valid로 분할 후 저장
train_new = df_train[:52683]
valid_new = df_train[52683:]

train_new.to_csv("train_new.csv", index=False)
valid_new.to_csv("valid_new.csv", index=False)

In [7]:
#test 데이터도 불러와서 전처리 후 저장
df_test = pd.read_csv("./input/dataset/test.csv")

for i in range(len(df_test)):
    df_test["text"][i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ ]","",df_test["text"][i]) #한국어 자음 포함해서 남겨둔 상태

df_random = pd.read_csv("./input/dataset/random_submission.csv")
df_test["label"] = df_random["label"]
df_test.to_csv("test_new.csv", index=False)
df_test.head()

C:\Users\user\AppData\Local\Temp\ipykernel_11572\32544924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["text"][i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ ]","",df_test["text"][i]) #한국어 자음 포함해서 남겨둔 상태


,ID,text,label
0,0,솔직히 우리나라 청년들도 불쌍하고 아재들도 불쌍하고 노인들도 불쌍하다 나라가 참,4
1,1,그만 보고싶네요 늙은애들은,0
2,2,더러운 개신교벌레 새퀴,6
3,3,근데전태수씨 사망이유가뭔가요그어떤기사에도 나오질않네요,0
4,4,태극기부대와 틀닭바퀴충들에게 순시리는 국모다 ㅉㅉ,4


In [8]:
# 안내문에 따라 install
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# Mecab을 토크나이저로 사용
# tokenizer = Mecab()
tokenizer = Okt()

In [9]:
print(len(train_new))
print(len(valid_new))
print(len(df_test))

52683
13180
13491


In [10]:
# 3개의 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=30)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)
# sequential : 시퀀스 데이터 여부. (True가 기본값)
# use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
# tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
# lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
# batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
# is_target : 레이블 데이터 여부. (False가 기본값)
# fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [11]:
train_data, valid_data, test_data = TabularDataset.splits(
        path='.', train='train_new.csv', validation='valid_new.csv', test='test_new.csv', format='csv',
        fields=[('ID',ID),('text', TEXT), ('label', LABEL)], skip_header=True)

In [12]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

52683
13180
13491


In [13]:
#잘 들어갔는지 확인
print(vars(train_data[0]))
print(vars(test_data[0]))

{'ID': '0', 'text': ['유소영', '비호감', '성형', '아줌마'], 'label': '1'}
{'ID': '0', 'text': ['솔직히', '우리나라', '청년', '들', '도', '불쌍하고', '아재', '들', '도', '불쌍하고', '노인', '들', '도', '불쌍하다', '나라', '가', '참'], 'label': '4'}


In [14]:
# 필드 구성 확인.
print(train_data.fields.items())

dict_items([('ID', <torchtext.legacy.data.field.Field object at 0x000001D959845E20>), ('text', <torchtext.legacy.data.field.Field object at 0x000001D959845E80>), ('label', <torchtext.legacy.data.field.Field object at 0x000001D959845DF0>)])


In [15]:
#각각의 단어집합 생성, 2번 이상 등장한 단어들 사용
TEXT.build_vocab(train_data, min_freq=2)
LABEL.build_vocab(train_data)
vocab_size = len(TEXT.vocab)
n_classes = 7
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 26695
클래스의 개수 : 7


In [16]:
print(type(TEXT.vocab.stoi)) #stoi로 단어와 각 단어의 정수 인덱스가 저장되어져 있는 딕셔너리 객체에 접근
from itertools import islice

for key, value in islice(TEXT.vocab.stoi.items(), 5):  # 첫 5개 데이터만 출력
    print(key, value)

<class 'collections.defaultdict'>
<unk> 0
<pad> 1
들 2
이 3
가 4


In [17]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 30

# torch version이 낮아서 GPU 사용 불가..
# DEVICE = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
# print(DEVICE)
DEVICE = torch.device('cpu')

In [23]:
# 토치텍스트는 모든 텍스트를 배치 처리하는 것을 지원하고, 단어를 인덱스 번호로 대체하는 BucketIterator를 제공한다.
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (train_data, valid_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False, sort=False) #sorted False 추가!!

In [24]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))

훈련 데이터의 미니 배치의 개수 : 824
검증 데이터의 미니 배치의 개수 : 206
테스트 데이터의 미니 배치의 개수 : 211


In [41]:
#첫 번째 미니 배치를 꺼냈으므로 재정의
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (train_data, valid_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False, sort=False) #sorted False 추가!!

In [42]:
train_value_counts = {
    0:0, 1:0, 2:0,3:0,4:0,5:0,6:0
}

valid_value_counts = {
    0:0, 1:0, 2:0,3:0,4:0,5:0,6:0
}

test_value_counts = {
    0:0, 1:0, 2:0,3:0,4:0,5:0,6:0
}

for i, batch in enumerate(train_iter):
    x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
    for tmp_y in y:
        train_value_counts[tmp_y.item()] += 1

for i, batch in enumerate(test_iter):
    x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
    for tmp_y in y:
        test_value_counts[tmp_y.item()] += 1

for i, batch in enumerate(val_iter):
    x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
    for tmp_y in y:
        valid_value_counts[tmp_y.item()] += 1

print(train_value_counts)
print(test_value_counts)
print(valid_value_counts)

{0: 6785, 1: 5664, 2: 6772, 3: 7105, 4: 3277, 5: 3141, 6: 19939}
{0: 1896, 1: 1941, 2: 1965, 3: 1990, 4: 1897, 5: 1904, 6: 1898}
{0: 1678, 1: 1443, 2: 1709, 3: 1790, 4: 841, 5: 758, 6: 4961}


In [40]:
# LSTM 모델 구현
class Net(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(Net, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        x, (ho,co) = self.lstm(x)
        h_t = x[:,-1,:] # 모든 문장을 거쳐서 나온 가장 마지막 단어의 출력 값
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

In [44]:
# 모델 훈련 함수
def train(model, optimizer, train_iter):
    """모델 학습"""
    model.train()
    for i, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)

        optimizer.zero_grad()

        logit = model(x)
        y= torch.tensor(y,dtype = torch.long,device =DEVICE)

        loss = F.cross_entropy(logit,y)
        loss.backward()
        optimizer.step()

In [45]:
# 모델 평가 함수
def evaluate(model, val_iter):
    """검증 데이터셋 평가"""
    my_list = []
    model.eval()
    batch_cor, total_loss = 0, 0

    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y= torch.tensor(y,dtype = torch.long,device =DEVICE)
        logit = model(x)

        loss = F.cross_entropy(logit, y)

        total_loss += loss.item()
        my
        batch_cor += (logit.max(1)[1] == y.data).sum() 

    size = len(val_iter.dataset)
    avg_loss = total_loss / size 
    avg_accuracy = 100.0 * batch_cor / size


    return avg_loss, avg_accuracy

In [47]:
a = torch.tensor([
    [1,2,3,4,5,6],
    [2,3,4,5,6,1]])
a.max(1)[1]

tensor([5, 4])

In [46]:
#모델 설계
model = Net(4, 128, vocab_size , 128, n_classes, 0.2).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [49]:
print(torch.__version__)
print(torchtext.__version__)

1.9.1+cpu
0.10.1


In [47]:
# 모델 훈련
best_val_loss = None
best_model = []

for e in range(1, EPOCHS + 1 ):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy= evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.5f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        torch.save(best_model.state_dict(), './txtclassification.pt')

C:\Users\user\AppData\Local\Temp\ipykernel_5868\3449882276.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y= torch.tensor(y,dtype = torch.long,device =DEVICE)


KeyboardInterrupt: 

In [37]:

def predict(model, test_iter):
    """학습된 모델로 정답 파일 예측"""
    model.eval()
    preds = []
    pred = []
    for batch in test_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y= torch.tensor(y,dtype = torch.long,device =DEVICE)

        logit = model(x)
        pred=torch.argmax(logit,axis= 1).tolist()
        preds.extend(pred)
    return preds 

In [ ]:
# model.load_state_dict(torch.load('./txtclassification.pt'))

In [34]:
pred = predict(best_model,test_iter)
len(pred)

/var/folders/v_/9p98qq6n7_bfq93v9pnc348c0000gn/T/ipykernel_10023/736960043.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y= torch.tensor(y,dtype = torch.long,device =DEVICE)


13491

In [35]:
submission = pd.read_csv('dataset/submission.csv')
submission.label = pred
submission.to_csv('lstm_ver05_ver02.csv',index= False)